In [16]:
from random import randint
from pygost import gost34112012256 as do_hash
import asn1


def sign(M, P,q, m, d,p,a,b):
    h = do_hash.new(M).digest()[::-1] # step 1
    #print("hash1:",h)
    h = int(h.encode('hex'), 16) #still step 1
    #print("hash2:",h)
    e = h % q # step 2 определяем е
    if e == 0:
        e = 1
    while True:
        k = randint(1,q) # step 3 
        C = k * P # step 4  - умножение числа к на точку P
        #print("This is dot: ",ZZ(C.xy()[0])) 
        r = ZZ(C.xy()[0]) % ZZ(q) # метод .xy возвращает кортеж, выбраем x, r=Xc(mod q)
        if r != 0:
            s = (r*d+k*e) % q # уравнение подписи
            if s != 0:
                break   
    return r,s # возвращаем подпись
    
def ASN1encode(P,Q,p,m,a,b,r,s,save_name):
    Qx,Qy=(Q).xy()
    Px,Py=P.xy()
    encoder = asn1.Encoder()
    encoder.start()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.enter(asn1.Numbers.Set)
    encoder.enter(asn1.Numbers.Sequence)
    encoder.write(b'\x80\x06\x07\x00', asn1.Numbers.OctetString) # идентификатор алгоритма ГОСТ
    encoder.write('gostSignKey', asn1.Numbers.UTF8String)
    encoder.enter(asn1.Numbers.Sequence) # значение открытого ключа Q
    encoder.write(int(Qx), asn1.Numbers.Integer) # x-координата Q
    encoder.write(int(Qy), asn1.Numbers.Integer) # y-координата Q
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence) # параметры криптосистемы
    encoder.enter(asn1.Numbers.Sequence) # параметры поля 
    encoder.write(p, asn1.Numbers.Integer) # простое число p, характеристика
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence) # параметры кривой
    encoder.write(a, asn1.Numbers.Integer) # коэффициент а кривой E 
    encoder.write(b, asn1.Numbers.Integer) # коэффициент b кривой E
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence) # образующая группы точке кривой
    encoder.write(int(Px), asn1.Numbers.Integer) # x-координата образующей точки P
    encoder.write(int(Py), asn1.Numbers.Integer) # y-координата образующей точки P
    encoder.leave()
    encoder.write(m, asn1.Numbers.Integer) # порядок группы m=q
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence) # подпись сообщения
    encoder.write(r, asn1.Numbers.Integer) # число r
    encoder.write(s, asn1.Numbers.Integer) #число s
    encoder.leave()
    encoder.leave()
    encoder.enter(asn1.Numbers.Sequence)
    encoder.leave()  
    encoder.leave()
    encoder.leave()
    encoded_bytes = encoder.output()
    f = open(save_name, "wb")
    f.write(encoded_bytes)
    f.close()
    
def verify(M, r, s, P, Q):
    q = P.order() # образующий, порядок
    if r > q or s > q: # step 1 
        return False
    h = do_hash.new(M).digest()[::-1] # step 2
    h = int(h.encode('hex'), 16) # step 2
    print hex(h)
    e = h % q # step 3
    if e == 0: 
        e = 1
    v = (ZZ(e) % q) ^ (-1) # step 4 
    z1 = s*v % q # step 5
    z2 = -r * v % q # step 5 
    C = z1 * P + z2 * Q # step 6
    R = ZZ(C.xy()[0]) % ZZ(q) # step 6
    return R == r # проверка подлинности подписи 

def ASN1decode(read_name_cert):
    f = open(read_name_cert, "rb")
    cert = f.read()
    f.close()
    decoder = asn1.Decoder()
    decoder.start(cert)
    decoder.enter()
    decoder.enter()
    decoder.enter()
    t,magic = decoder.read()
    t,string = decoder.read()
    decoder.enter()
    t,Qx = decoder.read()
    t,Qy = decoder.read()
    decoder.leave()
    decoder.enter()
    decoder.enter()
    t,p = decoder.read()
    decoder.leave()
    decoder.enter()
    t,a = decoder.read()
    t,b = decoder.read()
    decoder.leave()
    decoder.enter()
    t,Px = decoder.read()
    t,Py = decoder.read()
    decoder.leave()
    t,m = decoder.read()
    decoder.leave()
    decoder.enter()
    t,r = decoder.read()
    t,s = decoder.read()
    decoder.leave()
    decoder.leave()
    decoder.enter()
    decoder.leave()  
    decoder.leave()
    decoder.leave()
    return Qx,Qy,p,a,b,Px,Py,m,r,s

def verifyASN1(asn_file, check_file):
    Qx,Qy,p,a,b,Px,Py,m,r,s = ASN1decode(asn_file)
    E = EllipticCurve(FiniteField(p),[a,b])
    P = E(Px, Py) # генератор подргуппы
    Q = E(Qx, Qy) # ключ расшифрования   
    f = open(check_file, "rb")
    data = f.read()
    f.close()
    return verify(data, r, s, P, Q)
      
def signASN1(P,m,p,q,d,a,b,read_name, save_name):
    f = open(read_name, "rb")
    M = f.read()
    f.close() 
    r,s = sign(M, P,q, m, d,p,a,b)
    ASN1encode(P,P*d,p,m,a,b,r,s,save_name) # Q=d*P - ключ расшифрования
    
def main():
    print "Start!"
    p =  57896044625259982827082014024491516445703215213774687456785671200359045162371
    m =  28948022312629991413541007012245758222850495633896873081323396140811733708403 #m-порядок группы точек эллип. кривой E
    a =  -1
    b =  53956679838042162451108292176931772631109916272820066466458395232513766926866
    x =  12933162268009944794066590054824622037560826430730236852169234350278155715869
    y =  18786030474197088418858017573086015439132081546303111294023901101650919011383
    
    E = EllipticCurve(FiniteField(p),[a,b])
    #print(E)
    P = E(x, y) # обозначаем точку P
    #print("This is dot",P,"end")
    q = P.order() # m=nq, где n==1 какой-то малый сомножитель; q-порядок циклической подгруппы группы точек эллип. кривой E,и она равна m
    print q
    #print("This is q",q)
    d = randint(2,q) # личный ключ, ключ шифрования 
    print "Here"
#     read_name = raw_input("Enter filename to sign\n")
#     save_name = raw_input("Enter filename where to save signature\n")

    read_name = "C:\\Users\\Elvin\\Documents\\test12\\Kot.jpg"
    save_name = "C:\\Users\\Elvin\\Documents\\test12\\sign.asn1"
    print "Here2"
    signASN1(P,m,p,q,d,a,b,read_name, save_name)
    print verifyASN1(save_name, read_name)

main()

Start!
28948022312629991413541007012245758222850495633896873081323396140811733708403
Here
Here2
0x6c1b944b4e3fc86717d132d9ed541adb417a7572df3c5de4ac329f718a663be5L
True
